# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [18]:
import pandas as pd
import numpy as np
from scipy import stats

In [19]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [20]:
n_b, n_w = data.race.value_counts()
print('Number of black-sounding resumes: ', n_b)
print('Number of white-sounding resumes: ', n_w)

Number of black-sounding resumes:  2435
Number of white-sounding resumes:  2435


In [21]:
# number of callbacks for black-sounding names
callbacks_b = sum(data[data.race=='b'].call)
callbacks_w = sum(data[data.race=='w'].call)

In [22]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-info">
<p>
****Answers to Q1 and Q2 here:****
</p> 
<p>
Q1: What test is appropriate for this problem? Does CLT apply?
</p>
<p>
The appropriate test for this problem is a two-sample t-test comparing: resumes with black-sounding names and the resumes with white-sounding names. The Central Limit Theorem (CLT) applies in this case as the number of black-sounding resumes (2435) and white-sounding resumes (2435) are both in excess of >30, which is the minimum sample size for the Central Limit Theorem to hold. A t-test instead of a z-test is appropriate in this situation given that the sample variances are not known to be representative of the population variances. 
</p>
<p>
Q2: What are the null and alternate hypotheses?
</p>
<p>
The Null Hypothesis (Ho) states that the mean callback rate (m1) for resumes with black-sounding names is equal to the mean callback rate (m2) for resumes with white-sounding names. Stated mathematically:
Ho: m1 - m2 = 0
</p>

<p>
The Alternative Hypothesis (H1) states that the mean callback rate for resumes with black-sounding names is not equal to the mean callback rate for resumes with white-sounding names. Stated mathematically:
Ho: m1 - m2 != 0
</p>

</div>

In [23]:
w = data[data.race=='w'].call
b = data[data.race=='b'].call

<div class="span5 alert alert-info">
<p>
Q3: See below for computations for Margin of Error, Confidence Interval and p-value.
</p>
</div>

#### Bootstrap Replicate Approach

In [24]:
# Compute callback rate for all resumes: cbr_mean_all
cbr_mean_all = np.mean(data.call)
print('Mean callback rate all for all resumes: ', cbr_mean_all)

# Compute Empirical Difference in Mean Callback Rates: empirical_diff_means
empirical_diff_means = np.mean(w) - np.mean(b)
print('Empirical difference between callback rates: ', empirical_diff_means)

Mean callback rate all for all resumes:  0.08049281686544418
Empirical difference between callback rates:  0.03203285485506058


In [25]:
# Generated shifted arrays 
w_shifted = w - np.mean(w) + cbr_mean_all
b_shifted = b - np.mean(b) + cbr_mean_all

# Compute 1000 bootstrap replicates from shifted arrays
size = 1000
bs_replicates_w = np.empty(size)
bs_replicates_b = np.empty(size)
for i in range(size):
    bs_replicates_w[i] = np.mean(np.random.choice(w,len(w)))
    bs_replicates_b[i] = np.mean(np.random.choice(b,len(b)))

# Compute difference of means: bs_replicates
bs_replicates = bs_replicates_w - bs_replicates_b
# Compute and print p-value: p
p = np.sum(bs_replicates == empirical_diff_means) / size
print('p-value, bootstrap approach: {:5f}', p)

p-value, bootstrap approach: {:5f} 0.023


In [26]:
# Compute the 95% confidence interval, bootstrap approach
conf_int = np.percentile(bs_replicates, [2.5, 97.5])

# Compute Margin of Error, Bootstrap Replicate Approach
margin_of_error = 1.96 * bs_replicates.std()
print('The 95% confidence interval is from ', conf_int[0], ' to ', conf_int[1], '.')
print('The margin of error, as computed from the bootstrap replicates, is: ', margin_of_error)

The 95% confidence interval is from  0.0164271069691  to  0.046406570822 .
The margin of error, as computed from the bootstrap replicates, is:  0.0147304294339


#### Frequentist Approach

In [27]:
# Perform two-sample t-test
from scipy.stats import ttest_ind
t, p = ttest_ind(w, b)
print('p-value, two-sample t-test: {:5f}'.format(p))

# Compute 95% Confidence Interval, Frequentist Approach
import statsmodels.stats.api as sms
cm = sms.CompareMeans(sms.DescrStatsW(w), sms.DescrStatsW(b))
conf_int = cm.tconfint_diff(alpha=0.05, usevar='unequal')
print('The 95% confidence interval is from ', conf_int[0], ' to ', conf_int[1], '.')

# Compute Margin of Error
margin_of_error = conf_int[1] - empirical_diff_means 
print('The margin of error is ', margin_of_error)


p-value, two-sample t-test: 0.000039
The 95% confidence interval is from  0.016770673984  to  0.0472950344349 .
The margin of error is  0.0152621795798


<div class="span5 alert alert-info">
<p>
****Answers to Q4 and Q5 here:****
</p> 
<p>
Q4: Write a story describing the statistical significance in the context or the original problem.
</p>
<p>

Both the bootstrapping and the frequentist statistical approach produced p-values less than .05, such that we can reject the null hypothesis. The null hypothesis states that the mean callback rate for black-sounding resumes is equal to the mean callback rate for white-sounding resumes.  We reject the null hypothesis, and accept the alternate hypothesis that the mean callback rate for black-sounding resumes is NOT equal to the mean callback rate for white-sounding resumes.  This finding suggests that race/name is an important factor in callback success. 

</p>
<p>
Q5: Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
</p>

<p>

The analysis suggests that race/name is ONE important factor in callback success, but not necessarily the most important factor.  Sex/Gender, for example, could be another important factor in callback success that is not accounted for in this analysis. To account for Sex/Gener, I would conduct an additional two-sample t-test comparing the mean callback rates for men and women.  Additional analysis comparing the mean callback rates between black men, black women, white men and white women could also be conducted.

</p>

</div>